In [1]:
import import_ipynb

In [4]:
from ParsingMethods import parseCSV

all_contributions, entireText, timestamps, all_folders = parseCSV('nets_contributions17.csv')

('i', 1717)
('nt', 376)
('is', 355)
('2', 308)
('for', 307)
('class', 296)
('question', 295)
('would', 290)
('1', 271)
('best', 266)
('m', 256)
('get', 246)
('one', 231)
('a', 226)
('what', 224)
('code', 217)
('file', 210)
('could', 205)
('time', 203)
('b', 202)
('if', 201)
('know', 201)
('node', 196)
('nodes', 195)
('salman', 192)
('page', 186)
('url', 186)
('in', 179)
('number', 165)
('gt', 164)
('start', 161)
('lt', 156)
('document', 150)
('two', 144)
('example', 142)
('mean', 141)
('the', 140)
('using', 138)
('said', 137)
('use', 136)
('correct', 131)
('table', 127)
('like', 124)
('new', 122)
('can', 121)
('3', 119)
('instead', 117)
('need', 116)
('graph', 115)
('also', 114)
('how', 114)
('part', 113)
('or', 112)
('explain', 112)
('people', 112)
('algorithm', 110)
('do', 110)
('run', 109)
('list', 109)
('dfs', 108)
('0', 108)
('different', 108)
('edges', 107)
('find', 105)
('since', 105)
('however', 105)
('are', 103)
('when', 102)
('way', 101)
('homework', 99)
('documents', 99)
('s

In [48]:
from ManipulationMethods import only_one_folder

# Cluster within just one folder
all_nodes = list(all_contributions.values())

folder = "hw0"

# Returns a list of all nodes where the question is contained in folder
folder_contributions = only_one_folder(folder, all_nodes)

In [49]:
from ClusteringMethods import kmeans_cluster
from ManipulationMethods import create_cluster_dicts
from ManipulationMethods import short_question_to_node_dict

# Should put optimal k here
k = 4

# Creates a dictionary that links the first 100 characters of the shortened question to the raw node
# so that it can be retrieved later on
short_question_to_node, long_question_to_node = short_question_to_node_dict(folder_contributions)

# Create a list of all short questions in order to cluster them
short_questions = []
for node in folder_contributions:
    short_questions.append(node.shortened_question)

# Returns a list where the index is equal to the cluster label for the question at the same index in short_questions
labels = kmeans_cluster(k, short_questions)

# Returns two dictionaies one that links the short question to its cluster and one that links
# a cluster number to a list of questions
cluster_to_question, question_to_cluster = create_cluster_dicts(labels, short_questions)

for cluster in cluster_to_question:
    for question in cluster_to_question[cluster]:
        if question:
            print(short_question_to_node[question[0:100]].raw_question)
    print("\n\n\n\n\n\n\n\n\n\n\n\n\n")

ValueError: Found array with 0 sample(s) (shape=(0, 26239)) while a minimum of 1 is required.

In [ ]:
from BERTMethods import create_bert_model

bert_models = {}
raw_questions_for_bert = {}

# We want to iterate through and then come up with a BERT model for each cluster
for cluster_number in cluster_to_question:
    # Strip all questions so that we can determine which are empty
    raw_questions_for_bert[cluster_number] = [short_question_to_node[x[0:100]].raw_question for x in cluster_to_question[cluster_number] if x.strip()]
    bert_models[cluster_number] = create_bert_model(raw_questions_for_bert[cluster_number])

# BERT models is now a dict where each value is of the form (bc, doc_vecs)

In [ ]:
from BERTMethods import find_k_similar_sentences
topk_per_question = {}

# topk_per_question becomes a dict of the form question: ranked similarities for all questions in that cluster
for cluster_number in raw_questions_for_bert:
    bert_model = bert_models[cluster_number]
    for question in raw_questions_for_bert[cluster_number]:
        final_scores = find_k_similar_sentences(bert_model[0], bert_model[1], 10, raw_questions_for_bert[cluster_number], question)
        if len(list(final_scores.values())) > 0:
            topk_per_question[question] = final_scores[question]

In [ ]:
from BERTMethods import cluster_bert_clusters

initial_bert_clusters = cluster_bert_clusters(topk_per_question, k)

final_bert_clusters = {}
final_bert_node_to_cluster = {}

for cluster in initial_bert_clusters:
    print(cluster)
    if len(initial_bert_clusters[cluster]) > 1:
        print(initial_bert_clusters[cluster])

for cluster in initial_bert_clusters.keys():  
    for question in initial_bert_clusters[cluster]:
        final_bert_clusters.setdefault(cluster, []).append(long_question_to_node[question[0:100]])
        final_bert_node_to_cluster[long_question_to_node[question[0:100]].shortened_question[0:100]] = cluster

# print(initial_bert_clusters)

In [ ]:
from ManipulationMethods import find_clusters_to_combine

groups = find_clusters_to_combine(all_contributions, final_bert_node_to_cluster, len(list(initial_bert_clusters.keys())), short_question_to_node)

In [ ]:
print(groups)

final_clusters = []

for curr_group in groups:
    curr_cluster = []
    for num in curr_group:
        if num in final_bert_clusters:
            for node in final_bert_clusters[num]:
                curr_cluster.append(node)
    if len(curr_cluster) > 1:
        final_clusters.append(curr_cluster)

In [ ]:
import csv

with open(('output.csv_%s' % folder),'w') as f:
    writer = csv.writer(f)
    for sublist in final_clusters:
        to_write = []
        for item in sublist:
            to_write.append(item.raw_question)
        writer.writerow(to_write)